In [ ]:
orig_path = '10'
proc_path = '10processed'
quality = 'left10p'

In [3]:
import os
import argparse
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import time
from progress.bar import Bar
import torchvision.transforms as transforms
from dataloader.EyeQ_loader import DatasetGenerator
from utils.trainer import train_step, validation_step, save_output
from utils.metric import compute_metric

import pandas as pd
from networks.densenet_mcf import dense121_mcs

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)

# Setting parameters manually for notebook environment
parser = argparse.ArgumentParser(description='EyeQ_dense121')
parser.add_argument('--model_dir', type=str, default='./result/')
parser.add_argument('--pre_model', type=str, default='DenseNet121_v3_v1')
parser.add_argument('--save_model', type=str, default='DenseNet121_v3_v1')
parser.add_argument('--crop_size', type=int, default=224)
parser.add_argument('--label_idx', type=list, default=['Good', 'Usable', 'Reject'])
parser.add_argument('--n_classes', type=int, default=3)
parser.add_argument('--epochs', default=20, type=int)
parser.add_argument('--batch-size', default=4, type=int)
parser.add_argument('--lr', default=0.01, type=float)
parser.add_argument('--loss_w', default=[0.1, 0.1, 0.1, 0.1, 0.6], type=list)

# Simulating command line arguments
args = parser.parse_args(args=[])

import fundus_prep as prep
import glob
import os
import cv2 as cv
from PIL import ImageFile
from tqdm import tqdm
ImageFile.LOAD_TRUNCATED_IMAGES = True
import h5py

def process(image_list, save_path):
    
    for image_path in tqdm(image_list):
        dst_image = os.path.splitext(image_path.split('/')[-1])[0]+'.png'
        dst_path = os.path.join(save_path, dst_image)
        if os.path.exists(dst_path):
            print('continue...')
            continue
        try:
            img = prep.imread(image_path)
            r_img, borders, mask = prep.process_without_gb(img)
            r_img = cv.resize(r_img, (800, 800))
            prep.imwrite(dst_path, r_img)
            # mask = cv.resize(mask, (800, 800))
            # prep.imwrite(os.path.join('./original_mask', dst_image), mask)
        except:
            print(image_path)
            continue

list_left10 = glob.glob('/opt/notebooks/{}/*_0_0.png'.format(orig_path)) + glob.glob('/opt/notebooks/{}/*_0_1.png'.format(orig_path))
save_left10 = prep.fold_dir('/opt/notebooks/{}'.format(proc_path))
process(list_left10, save_left10)

model = dense121_mcs(n_class=args.n_classes)

if args.pre_model is not None:
    loaded_model = torch.load('/opt/notebooks/DenseNet121_v3_v1.tar',map_location=torch.device('cpu') )
    model.load_state_dict(loaded_model['state_dict'])

model.to(device)

criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
# glob G:\ukb\fundus_nomh_processed png file 
import glob
label = glob.glob('/opt/notebooks/{}/*.png'.format(proc_path))
label = [os.path.basename(x) for x in label]
label = pd.DataFrame(label, columns=['image'])
label['quality'] = 0
label['DR_grade'] = 0
# to csv with index
label.to_csv('/opt/notebooks/{}.csv'.format(proc_path), index=True)
# Images Labels
test_images_dir = "/opt/notebooks/{}/".format(proc_path)
label_test_file = "/opt/notebooks/{}.csv".format(proc_path)

# options
cudnn.benchmark = True

transformList2 = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

transform_list_val1 = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
    ])

data_test = DatasetGenerator(data_dir=test_images_dir, list_file=label_test_file, transform1=transform_list_val1,
                             transform2=transformList2, n_class=args.n_classes, set_name='test')
test_loader = torch.utils.data.DataLoader(dataset=data_test, batch_size=args.batch_size,
                                          shuffle=False, num_workers=0, pin_memory=True)
outPRED_mcs = torch.FloatTensor()

model.eval()
iters_per_epoch = len(test_loader)
bar = Bar('Processing {}'.format('inference'), max=len(test_loader))
bar.check_tty = False
for epochID, (imagesA, imagesB, imagesC) in enumerate(test_loader):
    # imagesA = imagesA.cuda()
    # imagesB = imagesB.cuda()
    # imagesC = imagesC.cuda()

    begin_time = time.time()
    _, _, _, _, result_mcs = model(imagesA, imagesB, imagesC)
    outPRED_mcs = torch.cat((outPRED_mcs, result_mcs.data), 0)
    batch_time = time.time() - begin_time
    bar.suffix = '{} / {} | Time: {batch_time:.4f}'.format(epochID + 1, len(test_loader),
                                                           batch_time=batch_time * (iters_per_epoch - epochID) / 60)
    bar.next()
bar.finish()

# save result into excel:
save_file_name = os.path.join('/opt/notebooks/{}.csv'.format(quality))
save_output(label_test_file, outPRED_mcs, args, save_file=save_file_name)

hdf5_path = '/opt/notebooks/{}.hdf5'.format(quality)
df = pd.read_csv('/opt/notebooks/{}.csv'.format(quality),index_col=0)
images_dir = '/opt/notebooks/{}'.format(proc_path)  # 替换为实际的图像目录路径
df['prediction'] = df[['Good','Usable','Reject']].idxmax(axis=1)
df = df[df['prediction'] != 'Reject']
df.reset_index(drop=True, inplace=True)

# 创建一个HDF5文件
with h5py.File(hdf5_path, 'w') as hdf:
    # 创建数据集来存储图像名称
    image_names_ds = hdf.create_dataset('image_names', shape=(len(df),), dtype=h5py.string_dtype())
    
    # 创建一个组来存储图像
    img_group = hdf.create_group('images')

    for i, row in tqdm(df.iterrows(), total=len(df)):
        img_name = row['image_name']  # 使用实际的列名
        
        # 读取图像
        img_path = os.path.join(images_dir, img_name)
        img = cv.imread(img_path, cv.IMREAD_COLOR)
        
        if img is not None:
            # 将图像数据转换为适合存储的格式
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            img_data = np.array(img)
            
            # 存储图像数据
            img_dataset = img_group.create_dataset(img_name, data=img_data, compression="gzip", compression_opts=9)
            
            # 存储图像名称
            image_names_ds[i] = img_name
        else:
            print(f"Error loading image {img_name}")

In [26]:
%%bash
dx upload /opt/notebooks/left10p.hdf5 
dx upload /opt/notebooks/left10p.csv

100%|██████████| 881/881 [02:06<00:00,  6.98it/s]
